In [ ]:
from larcv import larcv
from sklearn.cluster import DBSCAN
import numpy as np

from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
input_file="/data/dayajun/toymodel/uboone/train_data/75_200.root"
iom=larcv.IOManager(0)
iom.add_in_file(input_file)
iom.initialize()

In [ ]:
entry =0
iom.read_entry(entry, True)
ev_image    = iom.get_data(larcv.kProductImage2D,"wire")
ev_roi      = iom.get_data(larcv.kProductROI,"rui")
ev_instance = iom.get_data(larcv.kProductImage2D,"segment_rui")

In [ ]:
image = ev_image.Image2DArray()[2]
image.threshold(10,0)
test = larcv.as_ndarray(image)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,12))
ax.imshow(test,origin="lower")
# ax.imshow(test)

In [ ]:
for x in range(512):
    for y in range(512):
        if test[x,y]>0:
            print  x,y,test[x,y]
            break
    else:
        continue
    break


In [ ]:
X=np.argwhere(test>0)

In [ ]:
# def get_width(xy):
#     x=(np.max(np.array(xy[:,1]))-np.min(np.array(xy[:,1])))/2.
#     y=(np.max(np.array(xy[:,0]))-np.min(np.array(xy[:,0])))/2.
#     return x,y

def get_width(xy):
    delta_x=(np.max(xy[:,1])-np.min(xy[:,1]))
    delta_y=(np.max(xy[:,0])-np.min(xy[:,0]))
    return (delta_x<10 and delta_y<10)

In [ ]:
db = DBSCAN(eps=4, min_samples=10).fit(X)
# db = DBSCAN(eps=10, min_samples=10, metric=mydistance).fit(test)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
fig,ax=plt.subplots(1,1,figsize=(8,6))
    
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    if (k>=0): print xy2bb(xy)
    
#Here we ask to skip the cluster if the width is smaller than 5 pixel on both x & y axis
#     if ((k != -1) and get_width(xy)): continue
    ax.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=5)

    xy = X[class_member_mask & ~core_samples_mask]
    ax.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=2)

ax.set_xlim(0,512)
ax.set_ylim(0,512)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
def xy2bb(xy):
    tl=[np.min(xy[:,1]), np.max(xy[:,0])]
    tr=[np.max(xy[:,1]), np.max(xy[:,0])]
    bl=[np.min(xy[:,1]), np.min(xy[:,0])]
    br=[np.max(xy[:,1]), np.min(xy[:,0])]
    
    
    return tl,tr,bl,br

In [ ]:
this_roi=larcv.ROI()
origin=larcv.Point2D(1,2)

this_imagemeta=larcv.ImageMeta()

this_roi.PdgCode(11)


In [ ]:
print origin.x, origin.y

In [ ]:
for x in xrange(10):
    for y in xrange(10):
        print x*y
        if x*y > 50:
            break
    else:
        continue  # only executed if the inner loop did NOT break
    break  # only executed if the inner loop DID break